(results)=
# Results

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
plt.style.use('paper')

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
from src.visualization.plot import track_plots, track_plot, plot, plot_parameters
from scipy.linalg import block_diag
from scipy.stats import norm, multivariate_normal
import seaborn as sns

In [ ]:
regressions = {}

regressions['wPCC'] = catalog.load("wpcc.updated.vmm_martins_simple.joined.regression")
regressions['KVLCC2'] = catalog.load("kvlcc2_hsva.updated.vmm_martins_simple.joined.regression")

In [ ]:
stds/parameters

Hydrodynamic deriviatives in the selected Simplified Abkowitz model are shown in [tab](tab_parameters_X), [tab](tab_parameters_Y) and [tab](tab_parameters_N). The values are also shown in [fig](parameters). The standard deviation of the regressed parameters is small and hardly visible in this figure. The $N_{T\delta}$ for wPCC has the largest standard deviation, which is about 11% of its parameter value.

```{glue:figure} tab_parameters_X
:name: "tab_parameters_X"

Identified parameters in the Simplified Abkowitz model for wPCC and KVLCC2.
```

```{glue:figure} tab_parameters_Y
:name: "tab_parameters_Y"

Identified parameters in the Simplified Abkowitz model for wPCC and KVLCC2.
```

```{glue:figure} tab_parameters_N
:name: "tab_parameters_N"

Identified parameters in the Simplified Abkowitz model for wPCC and KVLCC2.
```

```{glue:figure} parameters
:name: "parameters"

Identified parameters in the Simplified Abkowitz model for wPCC and KVLCC2 including error bars with standard deviation.
```

[table](cov_X), [table](cov_Y), [table](cov_N) show the covarariance matrixes from the regression.

```{glue:figure} cov_X
:name: "cov_X"

Regression covariance matrix (*10^9) in the X-equation of Simplified Abkowitz model for KVLCC2.
```

```{glue:figure} cov_Y
:name: "cov_Y"

Regression covariance matrix (*10^9) in the Y-equation of Simplified Abkowitz model for KVLCC2.
```

```{glue:figure} cov_N
:name: "cov_N"

Regression covariance matrix (*10^9) in the N-equation of Simplified Abkowitz model for KVLCC2.
```



In [ ]:
parameters = pd.DataFrame()
stds = pd.DataFrame()

for ship, regression in regressions.items():
    parameters[ship] = regression.parameters['regressed']
    
    std = dict(regression.model_X.bse)
    std.update(regression.model_Y.bse)
    std.update(regression.model_N.bse)
    stds[ship] = pd.Series(std)
    
    
fig,ax=plt.subplots()
parameters.plot.bar(ax=ax, yerr=stds)

glue("parameters", fig, display=False)

In [ ]:
mask_X = parameters.apply(lambda x:x.name[0]=='X', axis=1)
parameters_X = parameters.loc[mask_X].transpose()
glue("tab_parameters_X",parameters_X)

In [ ]:
mask_Y = parameters.apply(lambda x:x.name[0]=='Y', axis=1)
parameters_Y = parameters.loc[mask_Y].transpose()
glue("tab_parameters_Y", parameters_Y)

In [ ]:
mask_N = parameters.apply(lambda x:x.name[0]=='N', axis=1)
parameters_N = parameters.loc[mask_N].transpose()
glue("tab_parameters_N", parameters_N)

In [ ]:
regression = regressions['KVLCC2']
models = {'X':regression.model_X, 
          'Y':regression.model_Y, 
          'N':regression.model_N}

fig,axes=plt.subplots(ncols=3)
covs = {}

for ax, (dof,model) in zip(axes, models.items()):
    keys = model.params.keys()
    cov = pd.DataFrame(model.cov_HC0, index=keys, columns=keys)
    covs[dof] = cov
    sigmas = np.sqrt(np.diag(cov))
    corr = cov.divide(sigmas, axis=0).divide(sigmas, axis=1)
    
    
    grid = sns.heatmap(corr.abs(), cmap='gray_r', ax=ax)
#plt.tight_layout()
glue("correlation", fig, display=False)

In [ ]:
for dof,cov in covs.items():
    glue(f"cov_{dof}", (cov*10**9).round(decimals=2))